In [1]:
# !wget https://github.com/mesolitica/llama2-embedding/raw/main/test-set/c.cari.com.my-dataset-bge-test.sample.json
# !wget https://huggingface.co/datasets/mesolitica/embedding-pair-mining/resolve/main/c.cari.com.my-dataset-bge-test.jsonl

In [2]:
import json
import openai
from tqdm import tqdm

In [3]:
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

A new version of the following files was downloaded from https://huggingface.co/mesolitica/llama2-embedding-1b-8k:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


[2023-10-10 22:55:41,155] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-10-10 22:55:41.689276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 22:55:42.430710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
_ = model.cuda()

In [5]:
with open('c.cari.com.my-dataset-bge-test.sample.json') as fopen:
    rev_data = json.load(fopen)

In [6]:
vectors = {}
for k, v in tqdm(rev_data.items()):
    try:
        padded = tokenizer([k],return_tensors = 'pt', padding = True)
        for k_ in padded:
            padded[k_] = padded[k_].cuda()

        vectors[k] = model.encode(padded).cpu().detach().numpy()[0]
    except:
        pass

100%|██████████| 15545/15545 [04:22<00:00, 59.15it/s]


In [7]:
no_string = {no: k for no, (k, v) in enumerate(vectors.items())}
string_no = {v: k for k, v in no_string.items()}
no_string[0]

'回复 5# 当猫爱上狗  我是27号要用的， 我想应该是来不及了。。真的是很谢谢你的回复。。'

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [9]:
line = 0
positive, negative = [], []
with open('c.cari.com.my-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        v_query = vectors.get(query)
        if v_query is None:
            continue
        v_query = np.array(v_query).reshape((1, -1))
        for s in l['pos']:
            v_s = vectors.get(s.strip())
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            positive.append(cosine_similarity(v_query, v_s)[0, 0])
        
        for s in l['neg']:
            v_s = vectors.get(s.strip())
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            negative.append(1 - cosine_similarity(v_query, v_s)[0, 0])
            
        line += 1
        if line >= 1000:
            break

999it [00:07, 137.31it/s]


In [16]:
np.mean(positive), np.mean(negative)

(0.74685395, 0.7647963317331168)

In [11]:
np_vectors = np.array(list(vectors.values()))
np_vectors.shape

(15545, 1536)

In [12]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}
total = 0
with open('c.cari.com.my-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        query_no = string_no.get(query)
        if query_no is None:
            continue
        for s in l['pos']:
            s = s.strip()
            v_s = vectors.get(s)
            s_no = string_no.get(s)
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            argsort = np.argsort(cosine_similarity(v_s, np_vectors)[0])[::-1]
            for k in tops.keys():
                if s_no in argsort[:k]:
                    k_ = k + 1
                else:
                    k_ = k
                if query_no in argsort[:k_]:
                    tops[k] += 1
            total += 1

7087it [08:05, 14.58it/s]  


In [13]:
tops, total

({1: 984, 3: 2261, 5: 2954, 10: 3885}, 11324)

In [15]:
for k, v in tops.items():
    print(k, v / total)

1 0.08689509007417874
3 0.19966442953020133
5 0.26086188625927237
10 0.3430766513599435
